1. 引用我自己寫的 helper function: query_from_db可以直接從DB獲取資料>
2. Import 會用到的 library
3. 定義了python request 連線的 header

In [36]:
# Import the pre-implement db query function

import os, sys
parent_dir = os.path.dirname(os.getcwd())
#os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(parent_dir)
from db_func import query_from_db
import html
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time

headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}

##### 將資料print出來的function，方便debug。

In [37]:
def print_util(urls, processor):
    for url in urls:
        try:
            print()
            temp = processor(url[0])
            print('News Url: \n{}\n'.format(url[0]))
            #print("Title: {}".format(temp['news_title']))
            #print("News: {}".format(temp['news']))
            for key in temp.keys():
                print('{}: {}'.format(key.capitalize(), temp[key]))
            print()
            print()
        except Exception as e:
            print()
            print(e)
            print('ERROR URL:')
            print(url)
            print()

### 選出10個在DB之中有最多資料的新聞來源：
```sql
SELECT news_source,COUNT(*) 
FROM news_db.news_rss_feeds
GROUP BY news_source
ORDER BY 2 DESC
LIMIT 11;
```

|  news_source    | COUNT(*) |
|------------------|----------|
| MSN              | 14204    |
| 自由時報         | 4997     |
| 大紀元           | 4394     |
| 經濟日報         | 3589     |
| ~新浪台灣新聞中心~ | ~3323~     |
| PCHOME           | 3213     |
| ETtoday          | 2585     |
| 新頭殼要聞       | 2117     |
| Rti 中央廣播電臺 | 2010     |
| Yahoo!奇摩股市   | 1976     |
| 公視新聞 |1796|  
最後因為新浪新聞有很多url是無效的，因此沒有放入 DB

### 1. 隨機選出50個Yahoo的新聞連結：
在開發時我是確保50個新聞連結都通過，但為了怕notebook篇幅太長，我改成五個來demo
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 50;
```

In [38]:
random_yahoo_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source LIKE '%Yahoo%' ORDER BY RAND()
LIMIT 100; """

yahoo_test_urls = query_from_db(random_yahoo_sql)

In [58]:
def yahoo_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    stop_footers = set(['全球疫情大流行', '今日最夯新聞流量前3名', '更多相關新聞', '更多 NOWnews 今日新聞 報導 ', '看更多 CTWANT 文章', '更多上報內容：', '更多 TVBS 報導', '《更多匯流新聞網CNEWS報導》', '更多社會相關新聞', '更多東森新聞報導', '原始連結', '更多鉅亨報導', '更多國際相關新聞', '***本文所提之內容僅供參考，不具投資建議，投資前應審慎評估風險，且自負盈虧***', '延伸閱讀：', '更多華視新聞報導', '更多匯流新聞網報導：', '【延伸閱讀】', '更多民視新聞報導', '更多財經相關新聞', '※【NOWnews 今日新聞】提醒您：', '更多健康2.0報導', '更多三立新聞網報導', '超人氣', '《TVBS》提醒您：', '《更多CNEWS匯流新聞網報導》', '更多新聞推薦', '更多新頭殼報導', '更多Heho健康網文章'])
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = html.unescape(title_category[0])
        if len(title_category) > 1:
            res_dict['news_category'] = html.unescape(title_category[1])
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = html.unescape(keywords_tag['content'])

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('time')
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            db_date_format = '%Y-%m-%d %H:%M:%S'
            res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = res
        except Exception as e:
            logging.error(e)
            print(e)
            pass

    article_body_tag = soup.find('article', attrs = {'itemprop':'articleBody'})
    caas_body_tag = soup.find('div', attrs = {'class': 'caas-body'})
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                if p.get('content'):
                    if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                        break
                    temp_content.append(html.unescape(p.get_text().strip()))
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif caas_body_tag:
        p_tags = caas_body_tag.find_all('p')
        a_tags = caas_body_tag.find_all('a')

        if p_tags:
            for index, p in enumerate(p_tags):
                if any([html.unescape(p.get_text()).startswith(stop_footer) for stop_footer in stop_footers]):
                    break
                temp_content.append(html.unescape(p.get_text()).strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    if temp_content:    
        # Capture the description start with 公告 to content
        if title_category[0][:4] == '【公告】':
            prefix = title_category[0]
        else:
            prefix = ''
        content = prefix + '\n'.join(temp_content)#.replace('。 ', '。\n')
        res_dict['news'] = html.unescape(content)
        return res_dict
    else:
        print('Yahoo url: {} did not process properly'.format(url))
        #content_parser_1.logger.error('Yahoo url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        return

In [61]:
print('Yahoo 新聞：')
print_util([['https://tw.news.yahoo.com/%E8%91%A1%E8%90%84%E7%8E%8B%E4%BB%8A%E5%B9%B4%E7%87%9F%E9%81%8B%E6%8B%9A%E9%9B%99%E4%BD%8D%E6%95%B8%E6%88%90%E9%95%B7-%E7%B5%B1-%E5%85%A5%E8%82%A1%E5%8A%A9%E6%94%BB-123921889.html']], yahoo_content_processor)

Yahoo 新聞：

News Url: 
https://tw.news.yahoo.com/%E8%91%A1%E8%90%84%E7%8E%8B%E4%BB%8A%E5%B9%B4%E7%87%9F%E9%81%8B%E6%8B%9A%E9%9B%99%E4%BD%8D%E6%95%B8%E6%88%90%E9%95%B7-%E7%B5%B1-%E5%85%A5%E8%82%A1%E5%8A%A9%E6%94%BB-123921889.html

News_title: 葡萄王今年營運拚雙位數成長 統一入股助攻
News_keywords: 中央社記者韓婷婷, 統一企業, 曾盛麟
News_description: （中央社記者韓婷婷台北18日電）葡萄王董事長曾盛麟表示，去年受到疫情影響的子公司葡眾及上海葡萄王營運均見到重返成長，全年營運力拚雙位數成長，統一企業透過私募持有葡萄王8%股權，今年董監改選預計取得1至2席董事。
News_published_date: 2021-01-18 12:39:21
News: （中央社記者韓婷婷台北18日電）葡萄王董事長曾盛麟表示，去年受到疫情影響的子公司葡眾及上海葡萄王營運均見到重返成長，全年營運力拚雙位數成長，統一企業透過私募持有葡萄王8%股權，今年董監改選預計取得1至2席董事。
受到疫情影響去年子公司葡眾、上海葡萄王均受到影響，僅台灣葡萄王逆勢成長，靠著自有品牌與海外代工推動去年營業額逆勢成長24%，成為支撐去年整體營業額小滑不到1%的重要力道。
曾盛麟表示，整體營運在去年底見到顯著復甦，台灣葡萄王目前代工訂單滿載，增加很多新客戶，考慮啟動三班制24小時生產，包括粉末、顆粒及玻璃瓶等產品都滿載；上海葡萄王的訂單能見度也已從疫情時的45天回到90天，第1季看來有機會雙位數成長。現階段看到第1季整體營運算樂觀。
至於葡眾，曾盛麟指出，如果疫情沒有進一步負面發展，預期葡眾仍可穩定成長，目前擁有30多種產品，預計將陸續推出3至5個新產品，除了保健產品外，與統一企業合作也將推出2項民生必需的快銷品。
葡萄王去年底宣布以私募方式辦理現金增資發行普通股，引進統一企業為投資人，並於1月14日股東臨時會決議通過，統一企業以每股價格不超過新台幣170元及總價不超過20億1467萬元認購葡萄王生技發行的私募普通股，取得葡萄王生技1185

### 2. 隨機選出50個 MSN 的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 50;
```

In [14]:
random_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 100; """
msn_test_urls = query_from_db(random_msn_sql)

In [93]:

def msn_content_processor(url):
    res_dict = {}
    prefix = ['相關報導', '※', '＊', '更多三立新聞網報導', '延伸閱讀', '資料來源', '更多中時電子報精彩報導', '看了這篇文章的人，也']
    pattern = re.compile(r'^{}'.format('|'.join(prefix)))
    
    if url.startswith('https://www.msn.com/zh-tw/video/'):
        return
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        res_dict['news_title'] = html.unescape(title_tag.text.strip())
        
    
    category_tag = soup.find('div', attrs = {'class':'logowrapper'})
    if category_tag:
        if len(category_tag.get_text().strip().split('\n')) == 2:
            _, category = category_tag.get_text().strip().split('\n')
            res_dict['news_category'] = html.unescape(category)
            
    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = html.unescape(description_tag['content'])

    time_tag = soup.find('div', attrs = {'class': 'timeinfo-txt'})
    if time_tag:
        date_time_tag = time_tag.find('time')
        try:
            d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.000Z") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('MSN date error, url: {}'.format(url))
    
    article_body_tag = soup.find('section', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'content'})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    elif article_body_tag_2:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(p.text.strip()))
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(html.unescape(div.text.strip()))
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(html.unescape(a.get_text().strip()))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
        
    if len(temp_content):
        content = '\n'.join(temp_content)
        res_dict['news'] = html.unescape(content)

    if not res_dict or 'news' not in res_dict: 
        #content_parser.logger.error('MSN url: {} did not process properly'.format(url))
        #content_parser.errors['process_empty_content_(rss_id)'].append([rss_id, url])
        print('MSN url: {} did not process properly'.format(url))
        return 
    return res_dict

In [94]:
print('MSN 新聞：')
print_util([['https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27']], msn_content_processor)

MSN 新聞：

News Url: 
https://www.msn.com/zh-tw/money/topstories/%E5%8F%B0%E7%A9%8D%E9%9B%BB%E6%89%BF%E8%AB%BE%E5%8A%A0%E9%80%9F%E4%BE%9B%E7%B5%A6-%E5%88%B0%E5%BA%95%E8%BB%8A%E7%94%A8%E6%99%B6%E7%89%87%E7%BC%BA%E8%B2%A8%E7%BC%BA%E4%BB%80%E9%BA%BC-%E8%AA%BF%E7%A0%94-%E9%80%993%E7%94%A2%E7%B7%9A%E6%9C%80%E7%B7%8A%E7%BC%BA/ar-BB1d8oqq?li=BBqj0iS&amp;srcref=rss%27

News_title: 台積電承諾加速供給！到底車用晶片缺貨缺什麼？調研：這3產線最緊缺
News_category: 財經
News_description: 經濟部針對全球車用晶片缺貨問題，邀請台積電、聯電、力積電、世界先進等國內主要代工廠代表一起商討。
News_published_date: 2021-01-27 21:38:00
News_related_url: []
News_related_url_desc: []
News: 編按（2021.1.28 13:30更新）：針對車用晶片缺貨問題，TrendForce報告指出，自2018年起車市逐步疲軟，加上2020年受到疫情嚴重衝擊，使主要模組廠的備貨動能明顯不足。然而，2021年全球汽車市場正在復甦，預估整車銷售量將自去年的7,700萬輛回升至8,400萬輛，同時汽車在自動化、智慧化和電動化發展下，對於各種半導體元件的用量將大幅上升，然先前因車市需求疲軟導致車廠備貨量偏低，長短料的現象已嚴重影響車廠稼動率與終端整車出貨。
因此，近期IC供應鏈的缺貨現象，已從消費性電子與電腦資通訊產業（Consumer & ICT），逐步蔓延到工控與車載市場（Industrial & Automotive）。過往車用半導體市場主要以垂直整合製造（IDM）或Fab-lite（即IDM企業將部分製造業務轉由協力廠商代工，是資產輕量化的一種策略）生產為主，例如恩智浦（NXP）、英飛凌（Infineon）、意法半導體

### 3. 隨機選出50個自由時報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 50;
```

In [17]:
ltn_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 100; """
ltn_test_urls = query_from_db(ltn_msn_sql)

In [110]:
ltn_sql_null = """
SELECT nrf.news_url 
FROM news_rss_feeds nrf
LEFT JOIN news_contents nc ON nrf.news_rss_feeds_id = nc.news_rss_feeds_id
WHERE nrf.rss_source = 'LTN' and nc.news_published_date IS NULL and nrf.processed_success = 1"""
ltn_test_urls = query_from_db(ltn_sql_null)

OperationalError: (2003, "Can't connect to MySQL server on '35.229.244.40' (timed out)")

In [111]:
ltn_test_urls

,news_url
0,https://health.ltn.com.tw/article/breakingnews...
1,https://health.ltn.com.tw/article/breakingnews...
2,https://health.ltn.com.tw/article/breakingnews...
3,https://health.ltn.com.tw/article/breakingnews...
4,https://health.ltn.com.tw/article/breakingnews...
...,...
175,https://health.ltn.com.tw/article/breakingnews...
176,https://health.ltn.com.tw/article/breakingnews...
177,https://health.ltn.com.tw/article/breakingnews...
178,https://health.ltn.com.tw/article/breakingnews...


In [1]:
def ltn_content_processor(url):
    postfix = ["""一手掌握經濟脈動
    點我訂閱自由財經Youtube頻道""", 
    """☆民眾如遇同居關係暴力情形，可撥打113保護專線，或向各地方政府家庭暴力防治中心求助☆""", 
    """☆健康新聞不漏接，按讚追蹤粉絲頁☆更多重要醫藥新聞訊息，請上自由健康網""",
    """《自由開講》是一個提供民眾對話的電子論壇，不論是對政治、經濟或社會、文化等新聞議題，有意見想表達、有話不吐不快，都歡迎你熱烈投稿。文長700字內為優，來稿請附真實姓名（必寫。有筆名請另註）、職業、聯絡電話、E-mail帳號。本報有錄取及刪修權，不付稿酬；請勿一稿多投，錄用與否將不另行通知。投稿信箱：LTNTALK@gmail.com""",
    """「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。""",
    """"""]
    res_dict = {}
    #headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
        else:
            try:
                res_dict['news_title'], res_dict['news_category'] = title_tag.string.split('|')
            except:
                pass
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag_1 = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('meta', attrs = {'name': 'pubdate'})
    time_tag_3 = soup.find('span', attrs = {'class': 'time'})
    if time_tag_1:
        try:
            d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
    elif time_tag_3:
        try:
            d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_3.get_text(), "%Y-%m-%d %H:%M:%S")
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print('LTN date error {}, URL: {}'.format(e2, url))
                #content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))
                

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag2 = soup.find('div', attrs = {'class':'text boxTitle boxText'})
    article_body_tag3 = soup.find('div', attrs = {'class':'text'})
    temp_content = []
    links = []
    links_descs = []

    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag2:
        p_tags = article_body_tag2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag2.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag3:
        p_tags = article_body_tag3.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag3.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        print('LTN {}, did not process properly'.format(url))
        #content_parser.logger.error('LTN url: {} did not process properly'.format(url))
        return
    return res_dict

In [17]:
print('自由時報')
print_util([['https://news.ltn.com.tw/news/world/breakingnews/3417111', '']],ltn_content_processor)

自由時報

News Url: 
https://news.ltn.com.tw/news/world/breakingnews/3417111

News_title: 媽媽氣炸！女兒上學疑遭虐全身29個針孔 老師辯「她夢到被打針」
News_category: 國際
News_fb_app_id: 140490219413038
News_keywords: 虐童,幼稚園老師,針孔
News_description: 中國陝西日前發生一起疑似虐童案，一名3歲女童自幼稚園放學回家後，媽媽竟發現女兒身上竟詭異多出29個針孔，一問之下女兒才說被老師打針，讓媽媽氣得報警處理。綜合中媒報導，陝西西安一名3歲楊姓女童本月14日自當地幼稚園「大興郡幼兒園」放學後，跟媽媽說自己腳痛，還說自己在學校被老師打針了，並表示幼稚園內只有她一個小朋友被老師打針。楊母一聽，隨即致電幼稚園老師是否園方有為小孩打針，老師否認此事，還稱女童是作夢夢到自己被打針。
News_published_date: 2021-01-20 12:26:51
News_related_url: []
News_related_url_desc: []
News: 中國陝西日前發生一起幼稚園疑似虐童案，女童回家被媽媽發現身上有明顯出血點，帶去醫院檢查確認全身有29處針孔。（圖擷取自微博）
〔即時新聞／綜合報導〕中國陝西日前發生一起疑似虐童案，一名3歲女童自幼稚園放學回家後，媽媽竟發現女兒身上竟詭異多出29個針孔，一問之下女兒才說被老師打針，讓媽媽氣得報警處理。
綜合中媒報導，陝西西安一名3歲楊姓女童本月14日自當地幼稚園「大興郡幼兒園」放學後，跟媽媽說自己腳痛，還說自己在學校被老師打針了，並表示幼稚園內只有她一個小朋友被老師打針。楊母一聽，隨即致電幼稚園老師是否園方有為小孩打針，老師否認此事，還稱女童是作夢夢到自己被打針。
楊母母起初也沒有太在意，但隔天（15日）竟發現女兒身上有多處明顯出血點，帶去醫院檢查後確認愛女全身上下有多達29處針孔，楊母氣得馬上在幼稚園與家長的聊天群組內發文，對老師提出質疑，但該老師卻已讀不回，園方也僅表示將介入了解此事，之後也沒有回應，楊母最後只能選擇報警。
「大興郡幼兒園」在警方介入調查後，一改先前態度，幼稚園園長出面聲稱「師德是幼兒園的底線，絕對是零容忍」，強調已調閱監視器，但沒有

In [29]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
content = MIMEMultipart()  #建立MIMEMultipart物件
content["subject"] = "Testing"  #郵件標題
content["from"] = "sfhsu29@gmail.com"  #寄件者
content["to"] = "sfhsu29@gmail.com" #收件者

  #郵件內容

In [14]:
import pickle
email = {}
email['addr'] = 'sfhsu29@gmail.com'
email['pw']= 'xztvygplcpiisfos'
with open('email_pw.config', 'wb') as f:
    pickle.dump(email, f)


In [32]:
import time
time.now()

AttributeError: module 'time' has no attribute 'now'

In [23]:
error = {}
error['type1'] = (50, 2, 10)
error['type2'] = (50, 30, 15)

In [30]:
for reason, ids in error.items():
    content.attach(MIMEText("{}: {}\n".format(reason, ids)))

In [31]:
import smtplib
with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
    try:
        smtp.ehlo()  # 驗證SMTP伺服器
        smtp.starttls()  # 建立加密傳輸
        smtp.login(email['addr'], email['pw'])  # 登入寄件者gmail
        smtp.send_message(content)  # 寄送郵件
        print("Complete!")
    except Exception as e:
        print("Error message: ", e)

Complete!


### 4. 隨機選出50個大紀元的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 50;
```

In [20]:
epoch_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 100; """
epoch_test_urls = query_from_db(epoch_test_sql)

In [21]:
def epoch_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ', 1)
        res_dict['news_title'] = title_category[0]
        
    
    category_tag = soup.find('meta', attrs = {'property':'article:section'})
    if category_tag:
        res_dict['news_category'] = str(category_tag['content'])
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag_2 = soup.find('div', attrs = {'class':'art-content'})
    #print(article_body_tag)
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag_2:
        p_tags = article_body_tag_2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag_2.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [22]:
print('大紀元：')
print_util(epoch_test_urls.values, epoch_content_processor)

大紀元：

News Url: 
https://www.epochtimes.com/b5/18/6/9/n10469975.htm

Title: 魔鬼在統治著我們的世界(15)：經濟篇(下)
News: 編按：「九評編輯部」兩年前的文章在今天引發更多共鳴，特此推薦，以饗讀者。
【大紀元2018年06月09日訊】第九章 經濟篇：魔鬼的誘餌（下）
目 錄
4. 公有制和計劃經濟是反天理的奴役制
1）公有制是魔鬼套在人民脖子上的枷鎖
2）計劃經濟是註定失敗的全能型權力經濟
5. 馬克思主義「剝削論」是顛倒善惡煽動仇恨的謬論
6. 絕對平均主義的毒根是恨和妒嫉
1）建立在恨和妒嫉基礎上的絕對平均主義
2）推廣經濟平權是走向共產主義的踏板
3）工會是共產邪靈安插進自由社會的釘子
7. 「共產主義理想」是引誘人類走向毀滅之途的誘餌
結語：重德才能「富而太平」
聽文章：
（聽更多新聞請至「聽紀元」）
上天造了人，賦予人智慧和體力，也給了人通過付出勞動得到正當物質回報的生活狀態，讓人獲得基本的生活保障。美國《獨立宣言》寫道：「我們認為下面這些真理是不言而喻的：造物者創造了平等的個人，並賦予他們若干不可剝奪的權利，其中包括生命權、自由權和追求幸福的權利。」[1]這些權利自然也包含對財產的擁有權和支配權。
而馬克思在《共產黨宣言》裡明確表示「共產黨人可以把自己的理論概括為一句話：消滅私有制」。[2]這就是要實行公有制；公有制必然要求實行計劃經濟，這種制度在本質上是違背天理、違背人性的奴役制。
美國反共運動先驅、思想家弗萊德‧施瓦茨在《本性難移的共產主義者》裡講了一個故事，說的是一個訪問者在前蘇聯和美國的汽車厰進行的兩段問答。[3]
問：「誰擁有這家工廠？」
工人答：「我們擁有。」
問：「那誰擁有工廠用的這塊地皮呢？」
工人答：「我們擁有。」
問：「誰擁有工廠製造出來的這些汽車呢？」
工人答：「我們擁有。」
在大樓外邊有一個很大的停車場，角落上停有三輛轎車。訪問者問，「誰擁有停車場的那幾輛汽車呢？」
工人答：「我們擁有它們，不過其中一輛歸廠長用、一輛歸黨委書記用、一輛歸祕密警察用。」
同一個訪問者又來到了美國的一個汽車厰，問工人同樣的問題。
問：「誰擁有這家工廠？」
工人答：「亨利‧福特擁有。」
問：「那誰擁有工廠用的這塊地皮呢？」
工人答：「亨利‧福特擁有。」
問：「誰擁有工廠製

### 5. 隨機選出50個經濟日報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 50;
```

In [23]:
udn_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 100; """
udn_test_urls = query_from_db(udn_test_sql)

In [24]:
import re
def udn_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category_lst = title_tag.string.split(' | ', 3)
        res_dict['news_title'] = title_category_lst[0]
        try:
            res_dict['news_category'] = title_category_lst[2]
        except:
            pass
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'shareBar__info--author'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.find('span').text, "%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d_temp = re.search('(\d{4}\-\d{2}\-\d{2}\s\d{2}\:\d{2})', time_tag.text).group(0)
                d1 = datetime.datetime.strptime(d_temp, "%Y-%m-%d %H:%M") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('Epoch date error {}'.format(e3))
                try:
                    d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%d %H:%M:%S") 
                    d1 -= datetime.timedelta(hours=8)
                    db_date_format = '%Y-%m-%d %H:%M:%S'
                    date_res = d1.strftime(db_date_format)
                    res_dict['news_published_date'] = date_res
                except Exception as e3:
                    print(e3)
                    content_parser.logger.info('Epoch date error {}'.format(e3))
    article_body_tag = soup.find('div', attrs = {'id':'article_body'})
    content_temp, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.get_text().strip() and p.get_text().strip() != 'facebook':
                    content_temp.append(p.get_text().strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs      
    content = '\n'.join(content_temp).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('udn url: {} did not process properly'.format(url))

    return res_dict


In [25]:
print_util(udn_test_urls.values, udn_content_processor)


News Url: 
https://money.udn.com/money/story/5599/5197098

Title: 川普離開白宮後首度發聲！向記者透露這項計畫
News: 美國前總統川普。法新社facebook
美國前總統川普離開白宮後於22日首度發聲，儘管他依然對未來計畫保持神秘，卻暗示一些行動正在進行中。
綜合外媒報導，川普在佛州棕梠攤川普國際高爾夫球俱樂部用晚餐時，被華盛頓觀察家報（Washington Examiner）記者問到關於未來的計畫，在其助理請記者離開前，川普簡短的回覆「我們會做些什麼，只是時間還沒到」。
川普於20日總統拜登就職典禮前不久抵達他的海湖莊園，隔日被拍到在俱樂部打高爾夫球。目前為止，川普乎沒有透露他卸任後的計畫。



News Url: 
https://money.udn.com/money/story/5599/5191716

Title: 賀拜登就職 文在寅盼攜手維持區域和平繁榮
News: 美國新總統拜登宣誓就職，韓國總統文在寅今天發出賀電，期盼以堅實韓美同盟維持朝鮮半島及區域和平繁榮，也表示希望早日與拜登會面。
韓聯社報導，文在寅於賀電中表示，拜登一直強調的和解與重建訊息引發美國人民共鳴，相信這位已做好準備的總統將實現美國的統合與繁榮。
他也期待拜登在2019冠狀病毒疾病（COVID-19，武漢肺炎）、氣候變遷、經濟危機等國際議題上展現領導風範。
文在寅強調，韓國將做為美國堅實同盟及可信賴的夥伴，陪伴拜登政府這次旅程。希望持續強化韓美同盟，維護朝鮮半島與區域的和平及繁榮。
文在寅也說，希望能在不久後與拜登面對面，開誠布公地對談。
文在寅同時在推特上以英文寫道「美國回來了」（America is back），祝賀拜登就任。



News Url: 
https://money.udn.com/money/story/5607/5188205

Title: 今日台股盤前重點掃描
News: 台股。記者杜建重／攝影facebook
一、鴻海打進Apple Car鏈 成長動能看旺
摩根大通與大和證券兩大外資看多鴻海（2317）電動車布局，直指鴻海已打入Apple Car供應鏈，成為Apple Car底盤、組件和次系統的關鍵供應商...看原文
二、晶圓雙雄爆量 占大盤1/5
半導體市況熱，晶圓代工產能供不應求

### 6. 隨機選出50個新浪台灣新聞中心的新聞連結： (skip this phase, too many error urls)
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50;
```

In [26]:
sina_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 20; """
sina_test_urls = query_from_db(sina_test_sql)

In [27]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [ ]:
import requests
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

In [ ]:
import random
# This data was created by using the curl method explained above
headers_list = [
    # Firefox 77 Mac
     {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
        # Chrome 83 Windows 
    {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
    }
]

def sina_content_processor(url):
    res_dict = {}
   # proxy = next(proxy_pool)
    r = requests.get(url, headers = random.choice(headers_list))
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-1].strip()
    
    
    # Sina did not have FB available
    #fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    #if fb_app_tag:
    #    res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']
        
    time_tag = soup.find('meta', attrs = {'name': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Sina date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'id':'article_content'})
    article_body_tag_2 = soup.find('div', attrs = {'class': 'pcont'})
    if article_body_tag:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content
    elif article_body_tag_2:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Sina url: {} did not process properly'.format(url))

    return res_dict


In [ ]:
print_util(sina_test_urls.values, sina_content_processor)

### 7. 隨機選出50個PCHOME的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 50;
```

In [28]:
pchome_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 100; """
pchome_test_urls = query_from_db(pchome_test_sql)

In [29]:

def pchome_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-2].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%S") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('PChome date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'calss':'article_text'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        a_tags = article_body_tag.find_all('a')
        if content:
            content = re.sub('(\n)+', '\n', content)
            content = re.sub(r'(相關新聞[\s\S]+)', '', content)
            res_dict['news'] = content 
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PChome url: {} did not process properly'.format(url))

    return res_dict

In [31]:
print_util(pchome_test_urls.values, pchome_content_processor)


News Url: 
https://news.pchome.com.tw/internation/cna/20210119/index-16110477456542518011.html

Title: 中國新一波疫情特徵：無症狀感染者特別多
News: 中國河北、黑龍江、吉林等省近來爆發的武漢肺炎疫情，出現大量的無症狀感染者，其中包括吉林的「一傳百」超級傳播者，成為這波疫情的特徵。且無症狀感染者的傳染性相對較強。
據官方通報，中國18日新增118例2019冠狀病毒疾病（COVID-19，武漢肺炎）確診病例，其中本土病例占106例，吉林、河北、黑龍江又各占43、35、27例，北京1例；此外，無症狀感染者也新增91例，其中80例為本土病例。
環球時報報導，中國每天新增的無症狀感染者居高不下，加上吉林日前出現的「一傳百」超級傳播者也屬無症狀感染者，使得無症狀感染者成為中國這波疫情的特徵及防控重點。
河北省疾控中心對此表示，大量的無症狀感染者，主要來自密切接觸者、疫情調查、暴露人群，以及曾赴疫情傳播區旅行和居住等4類人群。
同時，無症狀感染者也具有傳染性，尤其是處於潛伏期的無症狀感染者，在發病前1至2天和發病初期，傳染性都相對較強。但他們因為沒有任何明顯症狀，很少主動就醫，病毒傳播力卻不遜於確診病例，成為中國這波疫情防控的一大挑戰。
北京佑安醫院感染科副主任醫師李侗表示，無症狀感染者的呼吸道病毒並不一定比有症狀病例少，傳播能力取決於自身病毒量多少及飛沫傳播力。像是吉林的超級傳播者就是一名培訓講師，即使不咳嗽，說話也能散播病毒。
吉林省疾病預防控制中心應急辦公室主任趙慶龍則說，吉林出現黑龍江輸入病例後，第一時間開展大規模篩查，這使吉林的病例都處於感染早期，其中無症狀感染者比較多，但預料會逐漸轉為確診病例。
一名中國匿名專家認為，中國最近頻頻出現無症狀感染者，是因官方透過更及時的檢測，在還沒有發病的早期就發現這些無症狀感染者，是「檢測量大了」，並非感染者絕對人數上升。



News Url: 
https://news.pchome.com.tw/finance/cna_business/20210121/index-16112299782309722003.html

Title: 共享機車GoShare進駐高雄 累計全台6城市投入營運
News: Gogoro旗下的

### 8. 隨機選出50個Ettoday的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 50;
```

In [70]:
ettoday_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 100; """
ettoday_test_urls = query_from_db(ettoday_test_sql)

In [88]:
def ettoday_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    # Ettoday did not put fb page tag on the content
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    #if fb_page_tag:
        #res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('time', attrs = {'itemprop': 'datePublished'}) 
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                #content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))
    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e2:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get_text().strip(), "%Y年%m月%d日 %H:%M")
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e3:
                print(e3)
                #content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))

    

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links, links_descs = [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        
        if p_tags:
            for p in p_tags:
                # Ignore the image caption
                if p.find('strong'):
                    for match in p.find_all('strong', attrs = {'class': 'figcaption'}):
                        match.decompose()
                #    continue
                if p.text:
                    
                    temp_content.append(p.text.strip())
                print(p.text)
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip().replace('\u3000', ' '))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Ettoday url: {} did not process properly'.format(url))

    return res_dict

In [95]:
print_util([['https://house.ettoday.net/news/1889612?from=rss&redirect=1']], ettoday_content_processor)



▲台北市大樓都更容易嗎？（示意圖／取自免費圖庫Pixabay）

記者楊晴心／綜合報導

台北市大樓都更容易嗎？一位民眾發文表示，老家在台北市大安區精華地段，是一棟屋齡約35年的大樓，戶數約有70-80戶，然而最近公設一區一區開始壞，也沒人要修，便想知道台北市的大樓等的到都更嗎？多數網友認為，戶數多，難都更，賣一賣吧。

一名網友在PTT《home-sale》板上發文，表示自己老家在台北市大安區精華地段，是一棟屋齡約35年的大樓，該區房子成交價約落在2500-3000萬。原PO住的社區總戶數滿多，一棟戶數約70-80戶，也因為最近大樓公設慢慢出現問題，但卻沒有人想修，於是原PO便上網詢問網友，究竟是要等都更還是直接賣房？


▲多數人認為，戶數多的大樓要都更很困難。（示意圖／取自免費圖庫Pexels）

貼文一出，許多網友表示，35年大樓在台北根本不算老，「台北還有不少更爛更老的大樓，輪不到大安吧」、「35年還早啦，50年再來說都更」、「35在台北還不算老，一堆40年破公寓還硬拖不都更」；也有網友建議將老家出租，「租別人等都更，跟家人去別的地方租」。

另也有人分析，認為原PO住的大樓戶數太多，雖然大安區可以賭一下未來危老與都更的獎勵放寬，但短期內比較難「一堆公寓都還沒都更，誰會沒事去找七八十戶的大樓」、「就分配上來說，戶數多難度確實提高不少，土地持分少的都更較有機會成功」，最後也給出結論，「如果單純想參加都更，建議賣掉大樓買兩間公寓」。
News Url: 
https://house.ettoday.net/news/1889612?from=rss&redirect=1

News_title: 大安區35年大樓等不到都更？　除了屋齡還有「1重點」
News_category: ETtoday房產雲
News_fb_app_id: 146858218737386
News_keywords: 都更,危老,台北市,大安,房產
News_description: 台北市大樓都更容易嗎？一位民眾發文表示，老家在台北市大安區精華地段，是一棟屋齡約35年的大樓，戶數約有70-80戶，然而最近公設一區一區開始壞，也沒人要修，便想知道台北市的大樓等的到都更嗎？多數網友認.. (都更, 危老, 台北市, 大安, 房產, )
News_published_date: 202

### 9. 隨機選出50個Rti 中央廣播電臺的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 50;
```

In [51]:
rti_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 100; """
rti_test_urls = query_from_db(rti_test_sql)

In [52]:
def rti_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    reserved_keywords = ['Rti', '中央廣播電臺', 'Radio Taiwan International']
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ', 1)
        res_dict['news_title'] = title_category[0]
    category_tag = soup.find('div', attrs = {'class':'swiper-wrapper'})
    if category_tag:
        
        res_dict['news_category'] = category_tag.find('a', attrs = {'class':'active'}).get_text().strip()

    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        keyword_lst = [keyword for keyword in keywords_tag['content'].split(',') if keyword not in reserved_keywords]
        res_dict['news_keywords'] = ','.join(keyword_lst)

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if description_tags:
        res_dict['news_description'] = description_tags[1]['content']

    time_tag = soup.find('li', attrs = {'class': 'date'})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.text.strip(), "時間：%Y-%m-%d %H:%M:%S")
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                #content_parser.logger.info('RTI url: {} date not process properly, error message {}'.format(url, e2))

    article_body_tag = soup.find('article', attrs = {'class' : None})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('RTI url: {} did not process properly'.format(url))
    return res_dict

In [53]:
print_util(rti_test_urls.values, rti_content_processor)


News Url: 
https://www.rti.org.tw/news/view/id/2089794

Title: 撐住桃園、守住台灣 蘇揆：這一刻我們都是桃園人
News: 衛福部立桃園醫院群聚感染引發關注，行政院長蘇貞昌今天(22日)臨時決定參加桃園市防疫醫院聯防會議。蘇貞昌替部立桃醫的醫護同仁加油打氣，感謝部立桃醫承擔第一線防疫工作，並說在這個時刻，「我們都是桃園人」，他以桃園為榮，希望國人齊心合力撐住桃園、守住台灣。
部立桃園醫院群聚感染疫情升溫，行政院長蘇貞昌22日臨時新增行程，中午出席桃園市防疫醫院聯防會議，感謝桃園市長鄭文燦和其他協防醫院，並以視訊方式向部立桃醫的醫護同仁加油打氣。
蘇貞昌表示，在過去疫情最艱苦的一年，部立桃醫收治全國超過四分之一確診病患，提供隔離14000人的服務，對於資源不足的離島，桃園也提供防疫旅館收容服務。蘇貞昌表示，部立桃醫出狀況，第一線醫護仍然不懈怠，在這個緊急時刻「我們都是桃園人」，以桃園為榮，要齊心合力撐住桃園、守住台灣。他說：『(原音)這一刻我們都是桃園人，以桃園為榮，齊心合力撐住桃園、守住台灣。我們絕對沒有分彼此，去年在最需要時刻，桃園提供最大容量，甚至離島資源不足的時候，桃園提供收容。這一刻要給桃園更多鼓勵跟肯定。』
蘇貞昌感謝協助分倉分流的其他協防醫院、支援醫院和市區消毒的國軍六軍團33化兵群，以及桃園市長鄭文燦領導團隊守住疫情。
蘇貞昌表示，國外疫情不斷變化，台灣雖然守住1年，但是絕對不能大意，只有更努力、更合作，台灣才能安全、你我才能安全。蘇貞昌呼籲民眾一起挺桃園，讓台灣永遠是亂世中的福地。



News Url: 
https://www.rti.org.tw/news/view/id/2089491

Title: 研究：免疫系統至少能「記住」新型冠狀病毒6個月
News: 今天發布的新研究顯示，2019冠狀病毒疾病(COVID-19)患者康復後，由於體內細胞能「記住」病毒，因此可能可以避免至少6個月內再次染疫。
法新社報導，美國和瑞士研究人員對數十名染疫康復者調查結果發現，雖然他們的抗體可能隨時間流逝而消失，但體內仍會保有一定數量的特定「記憶B細胞」。
這些細胞可以記住病原體，面臨再度感染的情況時，也能促使免疫系統再生抗體來對抗病毒。
這份刊登在「自然」(Nature)期刊的研究總結：「記

### 10. 隨機選出50個公視新聞網的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 50;
```

In [54]:
pts_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 100; """
pts_test_urls = query_from_db(pts_test_sql)

In [128]:
def pts_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit(' | ', 1)
        res_dict['news_title'] = title_category[0].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if len(description_tags) > 1:
        res_dict['news_description'] = description_tags[1]['content']
        
    time_tag_2 = soup.find('div', attrs = {'class': 'maintype-wapper hidden-lg hidden-md'})
    time_tag = soup.find('time', attrs = {'class': None})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get_text(), "%Y-%m-%d %H:%M:%S") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e2, url))
                
        
    elif time_tag_2:
        date_tag = time_tag.find('h2')
        if date_tag:
            try:
                d1 = datetime.datetime.strptime(date_tag.text, "%Y年%m月%d日") 
                #d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e1:
                print(e1)
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e1, url))
    

    article_body_tag = soup.find('div', attrs = {'class':'article_content'})
    article_body_tag_2 = soup.find('article', attrs = {'class': 'post-article'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        if content:
            res_dict['news'] = content
    elif article_body_tag_2:
        content = article_body_tag_2.text.strip()
        if content:
            res_dict['news'] = content
    
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PTS url: {} did not process properly'.format(url))

    return res_dict

In [129]:
print_util([['https://news.pts.org.tw/article/509575', '']], pts_content_processor)


News Url: 
https://news.pts.org.tw/article/509575

News_title: 高收入國家大量購買、囤積新冠疫苗 譚德塞批「道德淪喪」 ｜ 公視新聞網 PNN
News_fb_app_id: 466487386878484
News_published_date: 2021-01-19 11:51:00
News: 各國加緊展開疫苗接種計畫，但世衛組織卻揭露，有不少高收入國家大量購買疫苗甚至囤貨，導致貧窮的國家無法獲得資源，秘書長譚德賽痛批這是道德淪喪。不過由國際專家組成的獨立調查小組指出，導致疫情擴散的最大原因是中國隱匿疫情、延誤通報，等於直接打臉秘書長譚德塞，因為他在初期不斷替中國護航。
隨著各國展開大規模疫苗接種計畫，截至目前全球49個高收入國家已經施打了3900萬劑。反觀相對貧窮、弱勢的國家，卻遲遲等不到疫苗，像是幾內亞只分配到25劑。世界衛生組織秘書長譚德塞看不下去，痛批這是道德淪喪。
世界衛生組織秘書長譚德塞批評，「我就直接地說了，這個世界正處於災難性道德淪喪的邊緣，而代價就是窮國的生命與生計。」
由世衛組織及其他國際機構主導建立的「疫苗全球取得機制」，目的讓156個會員國、全球近七成人口能平均分配疫苗。但近期世衛組織卻發現，歐美大國追求自身利益為優先，在批准緊急使用後，私下再與疫苗大廠簽訂單，甚至出現囤疫苗亂象。
世界衛生組織秘書長譚德塞指出，「這些亂象會延後COVAX分配疫苗，並與當初COVAX的目標背道而馳，全球恐將面臨囤疫苗的混亂市場，無法統一對抗疫情，進而造成社會及經濟的動盪。」
疫苗持續施打，調查病毒源頭的工作也在進行中。去年WHO組成的獨立專家小組，日前已抵達中國武漢。在最近一次的視訊報告中，調查小組指出，導致疫情擴散的最大原因，是中國隱匿疫情、延誤通報。
記者扎羅科斯塔斯報導，「調查小組認為中國地方和國家衛生部門，在去年一月疫情爆發時，應該採取更有力的公衛措施防堵疫情。」
此外，調查小組認為，WHO拖到1月22日才發布國際公衛警報，卻沒警告會有人傳人、無症狀感染等風險，也是導致全球大流行的原因。這項調查報告，等於直接打臉秘書長譚德塞，因為他在疫情爆發初期不斷替中國護航，還要求世界各國向中國學習。




### 11. 隨機選出50個新頭殼要聞的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50;
```

In [64]:
newstalk_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50; """
newstalk_test_urls = query_from_db(newstalk_test_sql)

In [67]:
def newstalk_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('NewsTalk date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')#, attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('NewsTalk url: {} did not process properly'.format(url))

    return res_dict

In [68]:
print_util(newstalk_test_urls.values, newstalk_content_processor)


News Url: 
https://newtalk.tw/news/view/2020-12-22/512583

Title: 去職震撼彈 林文淵：今天是我在大同最後一天
News: 大同董事長林文淵在11月接掌大同後，並在今天親上火線說明營運表現。林文淵卻爆出震撼彈，他指出，「今天是我在大同最後一天」。
百年大同公司經營權易主，林文淵在11月接任大同董事長，大同在今天召開記者會，林文淵親自上陣，先說明他到大同50天後所做的努力與改變。
不過，林文淵指出，大股東上週通知他將更換法人代表，原因是「時空環境不同」，雖然不懂更改原因，但他只能接受，將結束手邊工作。



News Url: 
https://newtalk.tw/news/view/2021-01-11/521894

Title: 明日開戰戴資穎直呼緊張找男雙對練
News: 超級1000系列泰國公開賽明日開戰，我國世界球后戴資穎睽違近10個月重返國際賽，今(11)日發文透露「感覺有點緊張」，也PO出到比賽場地訓練的影片，並找來我國第一男雙「麟洋配」王齊麟／李洋對打適應場地。
小戴去年3月15日(倫敦當地時間)擊敗中國一姊陳羽菲，抱回全英公開賽冠軍盃，並重返球后寶座，之後因新冠肺炎全球肆虐，國際賽開始停擺，去年10月的丹麥賽，小戴考慮疫情放棄衛冕，因此明天登場的泰國賽將是疫情後小戴的首戰。
小戴今天發文透露心情，「是不是真的很久沒有看我比賽了？ 我也一樣很久沒比賽了，感覺有點緊張。但不管怎麼樣，都希望可以享受回到場上的感覺。明天就有賽程囉，不要忘了看我打球。」
小戴也請來本站同樣要出賽的「麟洋配」對打，適應場地，「今天來到了主場訓練，感謝兩位的協助。明天要比賽啦」小戴首戰對手將是地主18歲小將Benyapa Aimsaard，雙方尚未有對戰紀錄。

延伸閱讀： 美軍方警告 拜登就職日須防範進一步威脅 NBA／表弟擊中詹皇頭部 惡意犯規遭驅逐出場 NBA／厄文因個人因素連缺3場 杜蘭特：我們支持他



News Url: 
https://newtalk.tw/news/view/2021-01-07/520004

Title: 「昨天有去探望鍾沛君」 柯文哲：PM2.5要嚴肅面對
News: 台北市國民黨籍議員鍾沛君日前檢查出右肺異常，經過開刀手術順利成功，市長柯文哲今(7)下午透露昨天有去探

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import jieba
import numpy as np
# 如果檔案內有一些編碼錯誤，使用 errors='ignore' 來忽略錯誤
with open("107.csv", encoding="big5", errors='ignore') as f:
    text = f.read()

# 設定使用 big5 斷詞
jieba.set_dictionary('dict.txt.big')
wordlist = jieba.cut(text)
words = " ".join(wordlist)
#文字雲造型圖片
mask = np.array(Image.open('picture.png')) #文字雲形狀
# 從 Google 下載的中文字型
font = 'SourceHanSansTW-Regular.otf'
#背景顏色預設黑色，改為白色、使用指定圖形、使用指定字體
my_wordcloud = WordCloud(background_color='white',mask=mask,font_path=font).generate(words)

plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()
#存檔
wordcloud.to_file('word_cloud.png')

In [33]:
import numpy as np
np.log(100)


4.605170185988092

In [35]:
import sys
sys.version

'3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'